In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setMaster("local").setAppName("applicaiton").set("spark.network.timeout", "10000000").set("spark.executor.heartbeatInterval", "10000000")
conf.set("spark.driver.memory", "5g") 
conf.set("spark.executor.memory", "2g")
conf.set("spark.cores.max", "2")
sc = SparkContext(conf=conf)

In [3]:
traffic_path = "C:\\Dataset\\Dodgers.data"
gamesInfo_path = "C:\\Dataset\\Dodgers.events"
traffic_data = sc.textFile(traffic_path)
gamesInfo_data = sc.textFile(gamesInfo_path)

In [4]:
gamesInfo_data.take(5)

['04/12/05,13:10:00,16:23:00,55892,San Francisco,W 9-8�',
 '04/13/05,19:10:00,21:48:00,46514,San Francisco,W 4-1�',
 '04/15/05,19:40:00,21:48:00,51816,San Diego,W 4-0�',
 '04/16/05,19:10:00,21:52:00,54704,San Diego,W 8-3�',
 '04/17/05,13:10:00,15:31:00,53402,San Diego,W 6-0�']

In [5]:
from datetime import datetime
import csv

In [6]:
# strptime is used to make the date string dateTime object.
def parseTraffic(row):
    DATE_TEMP = "%m/%d/%Y %H:%M"
    row = row.split(',')
    row[0] = datetime.strptime(row[0],DATE_TEMP)
    row[1] = int(row[1])
    return row[0],row[1]

In [7]:
traffiParsed = traffic_data.map(parseTraffic)

In [8]:
traffiParsed.take(5)

[(datetime.datetime(2005, 4, 10, 0, 0), -1),
 (datetime.datetime(2005, 4, 10, 0, 5), -1),
 (datetime.datetime(2005, 4, 10, 0, 10), -1),
 (datetime.datetime(2005, 4, 10, 0, 15), -1),
 (datetime.datetime(2005, 4, 10, 0, 20), -1)]

In [9]:
trafficTrend = traffiParsed.map(lambda x:(x[0].date(),x[1])).reduceByKey(lambda x,y:x+y)

In [10]:
# -x[1] the - represent the decending order.
trafficTrend.sortBy(lambda x:-x[1]).take(10)

[(datetime.date(2005, 7, 28), 7661),
 (datetime.date(2005, 7, 29), 7499),
 (datetime.date(2005, 8, 12), 7287),
 (datetime.date(2005, 7, 27), 7238),
 (datetime.date(2005, 9, 23), 7175),
 (datetime.date(2005, 7, 26), 7163),
 (datetime.date(2005, 5, 20), 7119),
 (datetime.date(2005, 8, 11), 7110),
 (datetime.date(2005, 9, 8), 7107),
 (datetime.date(2005, 9, 7), 7082)]

In [11]:
def parseGame(row):
    DATE_TEMP = "%m/%d/%y"
    row = row.split(',')
    row[0] = datetime.strptime(row[0],DATE_TEMP).date()
    return row[0],row[4]

In [12]:
gameInfoParsed = gamesInfo_data.map(parseGame)

In [13]:
gameInfoParsed.take(5)

[(datetime.date(2005, 4, 12), 'San Francisco'),
 (datetime.date(2005, 4, 13), 'San Francisco'),
 (datetime.date(2005, 4, 15), 'San Diego'),
 (datetime.date(2005, 4, 16), 'San Diego'),
 (datetime.date(2005, 4, 17), 'San Diego')]

In [14]:
#The code below demonstrate how to merge the two seperate RDD by using Join
joinedData = trafficTrend.leftOuterJoin(gameInfoParsed)
joinedData.take(10)

[(datetime.date(2005, 4, 10), (-288, None)),
 (datetime.date(2005, 4, 11), (5062, None)),
 (datetime.date(2005, 4, 14), (6423, None)),
 (datetime.date(2005, 4, 15), (6459, 'San Diego')),
 (datetime.date(2005, 4, 16), (6002, 'San Diego')),
 (datetime.date(2005, 4, 17), (5322, 'San Diego')),
 (datetime.date(2005, 4, 18), (5600, None)),
 (datetime.date(2005, 4, 19), (6049, None)),
 (datetime.date(2005, 4, 21), (5977, None)),
 (datetime.date(2005, 4, 22), (6038, None))]

In [15]:
joinedData.sortBy(lambda x:-x[1][0]).take(10)

[(datetime.date(2005, 7, 28), (7661, 'Cincinnati')),
 (datetime.date(2005, 7, 29), (7499, 'St. Louis')),
 (datetime.date(2005, 8, 12), (7287, 'NY Mets')),
 (datetime.date(2005, 7, 27), (7238, 'Cincinnati')),
 (datetime.date(2005, 9, 23), (7175, 'Pittsburgh')),
 (datetime.date(2005, 7, 26), (7163, 'Cincinnati')),
 (datetime.date(2005, 5, 20), (7119, 'LA Angels')),
 (datetime.date(2005, 8, 11), (7110, 'Philadelphia')),
 (datetime.date(2005, 9, 8), (7107, None)),
 (datetime.date(2005, 9, 7), (7082, 'San Francisco'))]

In [24]:
def checkGameDay(row):
    try:
        if row[1][1] == None:
            return (row[0], row[1][1],"RegularDay",row[1][0])
        else:
            return (row[0], row[1][1],"GameDay",row[1][0])
    except Exception as ex:
        return "Error" + ex

In [25]:
dailyTrendByGames = joinedData.map(checkGameDay)


In [26]:
dailyTrendByGames.sortBy(lambda x:+x[3]).take(10)

[(datetime.date(2005, 4, 10), None, 'RegularDay', -288),
 (datetime.date(2005, 10, 1), None, 'RegularDay', -260),
 (datetime.date(2005, 6, 28), 'San Diego', 'GameDay', -96),
 (datetime.date(2005, 7, 4), None, 'RegularDay', 328),
 (datetime.date(2005, 7, 12), None, 'RegularDay', 1204),
 (datetime.date(2005, 5, 23), None, 'RegularDay', 2173),
 (datetime.date(2005, 9, 17), None, 'RegularDay', 2426),
 (datetime.date(2005, 9, 10), 'San Diego', 'GameDay', 2851),
 (datetime.date(2005, 6, 27), 'San Diego', 'GameDay', 2907),
 (datetime.date(2005, 7, 10), None, 'RegularDay', 3518)]

In [44]:
# Get the average traffic counts of game day and of regular day.
dailySubCountByGame =  dailyTrendByGames.map(lambda x: (x[2],x[3]))
dailySubCountByGame.sortBy(lambda x:+x[1]).take(10)

[('RegularDay', -288),
 ('RegularDay', -260),
 ('GameDay', -96),
 ('RegularDay', 328),
 ('RegularDay', 1204),
 ('RegularDay', 2173),
 ('RegularDay', 2426),
 ('GameDay', 2851),
 ('GameDay', 2907),
 ('RegularDay', 3518)]

In [45]:
dailySubCountByGame = dailySubCountByGame.combineByKey(
    lambda val: (val,1),
    lambda acc,val:(acc[0]+val,acc[1]+1),
    lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])
)

In [46]:
dailySubCountByGame.map(lambda x:(x[0],x[1][0]/x[1][1])).collect()

[('GameDay', 5948.604938271605), ('RegularDay', 5411.329787234043)]